## Week 6 : Large Language Models
```
- Generative Artificial Intelligence (Fall semester 2023)
- Professor: Muhammad Fahim
- Teaching Assistant: Ahmad Taha
```
<hr>

## Contents
```
1. Transformers (Implementing a transformer)
2. Self-Attention
3. Multi-headed attention
4. Positional Encoding

```

<hr>


# Transformers

* [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf) -- Original paper on attention

![](http://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png)


In [1]:
import torch
from torch import nn
import torch.optim as optim
import pandas as pd
import numpy as np

from torch.utils.data import DataLoader, TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Transformer Encoder with Pytorch

In [2]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=32)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [3]:
encoder_layer

TransformerEncoderLayer(
  (self_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
  )
  (linear1): Linear(in_features=512, out_features=2048, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=2048, out_features=512, bias=True)
  (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
)

## Encoder

The encoder contains a self-attention layer – a layer that helps the encoder look at other words in the input sentence as it encodes a specific word. <br>
**The main goal is to efficiently encode the data**

![](http://jalammar.github.io/images/t/encoder_with_tensors.png)

## Self-Attention

**Keep in mind : The main goal is to encode the data in a much more efficient way** In other words is to create meaningful embeddings<br>
- As the model processes each word (each position in the input sequence), self attention allows it to look at other positions in the input sequence for clues that can help lead to a better encoding for this word.


**How does Self-Attention work?**

Steps:
1. For each word, we create a **`Query`** vector, a **`Key`** vector, and a **`Value`** vector.
  - What are the **`Query`** vector, a **`Key`** vector, and a **`Value`** vector? : They’re abstractions that are useful for calculating attention... They are a breakdown of the word embeddings
2. Calculating self-attention score from **`Query`** **`Key`** vector.
3. Divide the scores by 8 (This leads to having more stable gradients)
4. Pass the result through a softmax operation (softmax score determines how much each word will be expressed at this position)
5. Multiply each value vector by the softmax score
6. Sum up the weighted value vectors

### Step 1

For each word, we create a **`Query`** vector, a **`Key`** vector, and a **`Value`** vector.

![](http://jalammar.github.io/images/t/transformer_self_attention_vectors.png)

In [4]:
# simple sequence = I am here today
simple_sequence_embedding = torch.rand((4, 512))

# Create weight matrices
Wq = torch.normal(0,0.5, (512, 7))
Wk = torch.normal(0,0.1, (512, 7))
Wv = torch.normal(0,0.2, (512, 7))

# Create key, query and value for each word in the senetence
queries = simple_sequence_embedding.mm(Wq) # self.q(embedding[0])
keys = simple_sequence_embedding.mm(Wk)
values = simple_sequence_embedding.mm(Wv)

In [5]:
values

tensor([[-2.8819,  0.3868,  0.6122,  1.1638, -0.4031,  0.1574, -0.7762],
        [-2.7936, -1.8659,  0.0103,  0.2582, -0.1112, -2.6158, -2.6719],
        [-4.5350,  0.5986,  0.0890,  2.0842, -2.3219, -1.9934, -1.2703],
        [-5.2214,  1.0248, -0.0500,  1.9272,  1.1439, -2.5974, -1.6072]])

In [6]:
simple_sequence_embedding

tensor([[0.8522, 0.0400, 0.3084,  ..., 0.8247, 0.4982, 0.6945],
        [0.7480, 0.7605, 0.2476,  ..., 0.7656, 0.9916, 0.6474],
        [0.4129, 0.4805, 0.3385,  ..., 0.2986, 0.1851, 0.3807],
        [0.5022, 0.7521, 0.8176,  ..., 0.7204, 0.7519, 0.4126]])

## Step 2

Calculating self-attention score from **`Query`** and **`Key`** vector

In [7]:
scores = torch.mm(queries, keys.T)
scores

tensor([[  7.2688,  -2.7020,  10.7785,  -9.2198],
        [  1.0508,   3.0068,  10.7285, -15.1156],
        [  5.0860,   0.4926,   9.4564,   1.8765],
        [ -2.1986,  -5.0873,   1.2713,  -1.7891]])

## Step 3
Divide the scores by 8 (This leads to having more stable gradients)

In [8]:
scores = scores / 8
scores

tensor([[ 0.9086, -0.3377,  1.3473, -1.1525],
        [ 0.1314,  0.3759,  1.3411, -1.8894],
        [ 0.6357,  0.0616,  1.1821,  0.2346],
        [-0.2748, -0.6359,  0.1589, -0.2236]])

## Step 4

Pass the result through a softmax operation

In [9]:
scores = torch.nn.functional.softmax(scores, dim=1)
scores

tensor([[0.3372, 0.0970, 0.5229, 0.0429],
        [0.1735, 0.2216, 0.5818, 0.0230],
        [0.2526, 0.1422, 0.4361, 0.1691],
        [0.2330, 0.1624, 0.3595, 0.2452]])

## Step 5 & 6

* Multiply each value vector by the softmax score
* Sum up the weighted value vectors



In [10]:
scores.shape, values.shape

(torch.Size([4, 4]), torch.Size([4, 7]))

In [11]:
z = scores @ values
z

tensor([[-3.8382,  0.3065,  0.2519,  1.5901, -1.3117, -1.3544, -1.2540],
        [-3.8780,  0.0255,  0.1592,  1.5162, -1.4192, -1.7720, -1.5029],
        [-3.9859,  0.2666,  0.1865,  1.5655, -0.9368, -1.6409, -1.4018],
        [-4.0355,  0.2536,  0.1641,  1.5348, -0.6661, -1.7415, -1.4654]])

# Multi-headed attention

**GOAL**:
1. Expand the model’s ability to focus on different positions
2. Provide the attention layer multiple “representation subspaces”

**Attention with $N$ just means repeating self attention algorithm $N$ times and joining the results**


![](https://data-science-blog.com/wp-content/uploads/2022/01/mha_img_original.png)

**Multi-headed attention steps:**
1. Same as self-attention calculation, just n different times with different weight matrices
2. Condense the $N$ z metrices down into a single matrix by concatinating the matrices then multiply them by an additional weights matrix `WO`

Now the output z metrix is fed to the FFNN

In [12]:
from torch import Tensor
import torch.nn.functional as f


def scaled_dot_product_attention(query, key, value):
  temp = query.bmm(key.transpose(1, 2))
  scale = query.size(-1) ** 0.5
  softmax = f.softmax(temp / scale, dim=-1)
  return softmax.bmm(value)

## Now lets make attention head

In [13]:
import torch.nn as nn

class AttentionHead(nn.Module):
    def __init__(self, dim_in, dim_q, dim_k):
        super().__init__()
        # Fill in the missing parts of the constructor to initialize the query, key, and value linear transformations
        # dim_in is the input dimension, dim_q and dim_k are the output dimensions for the queries and keys/values respectively
        # Example: dim_in 512, dim_q = dim_k 64 in the paper
        self.q = nn.Linear(dim_in, dim_q)
        self.k = nn.Linear(dim_in, dim_k)
        self.v = nn.Linear(dim_in, dim_k)

    def forward(self, query, key, value):
        # Implement the forward pass by calling scaled_dot_product_attention function
        # You need to transform the query, key, and value using the linear transformations defined in __init__
        # Fill in with the correct method calls
        return scaled_dot_product_attention(self.q(query), self.k(key), self.v(value))

## Multi Head Attention

In [14]:
class MultiHeadToAttention(nn.Module):
    def __init__(self, number_of_heads, dim_in, dim_q, dim_k):
        super().__init__()
        # Initialize heads as multi-AttentionHead instances
        # Initialize linear to combine the outputs of all heads into a single output
        self.heads = nn.ModuleList([AttentionHead(dim_in, dim_q, dim_k) for _ in range(number_of_heads)])
        self.linear = nn.Linear(number_of_heads * dim_k , dim_in)

    def forward(self, query: Tensor, key: Tensor, value: Tensor):
        # Concatenate outputs from all heads and apply the final linear transformation
        out = [head(query, key, value) for head in self.heads]
        return self.linear(torch.cat(out, dim=-1))

## Positional Encoding

A way to account for the order of the words in the input sequence. A transformer adds a vector to each input embedding which helps it determine the position of each word. <br>
**Goal** : preserving information about the order of tokens  

Positional Encoding they can either be learned or fixed a priori.

Proposed approach from original paper : describe a simple scheme for fixed positional encodings based on sine and cosine functions

![](https://miro.medium.com/v2/resize:fit:640/format:webp/1*C3a9RL6-SFC6fW8NGpJg5A.png)

In [15]:
def position_encoding(seq_len, dim_model, device):
    # Define the position tensor 'pos' with dimensions appropriate for sequence length
    pos = torch.arange(seq_len, dtype=torch.float, device=device).reshape(1, -1, 1)

    # Define the dimension tensor 'dim' suitable for the model dimensions
    dim = torch.arange(dim_model, dtype=torch.float, device=device).reshape(1, 1, -1)

    # Calculate the phase using the position and dimension tensors
    phase = pos / (1e4 ** (dim / dim_model))

    # Return the sinusoidal position encoding
    return torch.where(dim.long() % 2 == 0, torch.sin(phase), torch.cos(phase))

## Encoder Feed Forward

In [16]:
def feed_forward(dim_input = 512, dim_feedforward = 2048):
  return nn.Sequential(nn.Linear(dim_input, dim_feedforward),
                       nn.ReLU(),
                       nn.Linear(dim_feedforward, dim_input)
                       )

## Encoder Residual

From the original paper the author implementation

In [17]:
class Residual(nn.Module):
  def __init__(self, sublayer, dimension, dropout = 0.1):
    super().__init__()
    self.sublayer = sublayer
    self.norm = nn.LayerNorm(dimension)
    self.dropout = nn.Dropout(dropout)

  def forward(self, *tensors):
    return self.norm(tensors[0] + self.dropout(self.sublayer(*tensors)))

## Putting all together on encoder side

![](http://jalammar.github.io/images/t/transformer_resideual_layer_norm_2.png)

## Putting the Encoder layer together

In [18]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, dim_model=512, num_heads=6, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        # Define dimensions for query and key based on model dimension and number of heads
        dim_q = dim_k = max(dim_model // num_heads, 1)

        # Initialize the MultiHeadAttention component with a residual connection and dropout
        self.attention = Residual(MultiHeadToAttention(num_heads, dim_model, dim_q, dim_k),
                              dimension=dim_model, dropout=dropout)

        # Initialize the feedforward component with a residual connection and dropout
        self.feed_forward = Residual(feed_forward(dim_model, dim_feedforward), dimension=dim_model, dropout=dropout)

    def forward(self, src):
        # Apply the attention mechanism to the source data
        src = self.attention(src, src, src)

        # Apply the feedforward network to the output of the attention mechanism
        return self.feed_forward(src)

## Putting together transfomer Encoder part

In [19]:
class TransformerEncoder(nn.Module):
    def __init__(self, num_layers=12, dim_model=512, num_heads=4, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        # Initialize a list of TransformerEncoderLayer instances
        self.layers = nn.ModuleList([TransformerEncoderLayer(dim_model, num_heads, dim_feedforward, dropout) for _ in range(num_layers) ])

    def forward(self, src):
        # Retrieve the sequence length and dimension from the source tensor
        seq_len, dimension = src.size(1), src.size(2)

        # Add position encoding to the source tensor
        src += position_encoding(seq_len, dimension, src.device)

        # Process each layer in the transformer encoder
        for layer in self.layers:
            src = layer(src)

        return src

# The Decoder Side

The encoder start by processing the input sequence. The output of the top encoder is then transformed into a set of attention vectors K and V. These are to be used by each decoder.


![](https://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png)

## Decoder layer

The “Encoder-Decoder Attention” layer works just like multiheaded self-attention, except it creates its Queries matrix from the layer below it, and takes the Keys and Values matrix from the output of the encoder stack.

**Task**: implement the decoder layer

In [20]:
class TransformerDecoderLayer(nn.Module):
    def __init__(
        self,
        dim_model: int = 512,
        num_heads: int = 6,
        dim_feedforward: int = 2048,
        dropout: float = 0.1,
    ):
        super().__init__()
        # init dim_q and dim_k as in the encoder
        dim_q = dim_k = max(dim_model // num_heads, 1)
        # Initialize the first self-attention layer with a residual connection
        self.attention_1 = Residual(MultiHeadToAttention(num_heads, dim_model, dim_q, dim_k),
                                       dimension=dim_model, dropout=dropout)

        # Initialize the second attention layer for interaction with the encoder output
        self.attention_2 = Residual(MultiHeadToAttention(num_heads, dim_model, dim_q, dim_k),
                                          dimension=dim_model, dropout=dropout)

        # Initialize the feed-forward network
        self.feed_forward = Residual(feed_forward(dim_model, dim_feedforward), dimension=dim_model, dropout=dropout)

    def forward(self, tgt: Tensor, memory: Tensor) -> Tensor:
        # Self-attention mechanism
        tgt = self.attention_1(tgt, tgt, tgt)

        # Cross-attention mechanism where the decoder attends to the encoder's output
        tgt = self.attention_2(tgt, memory, memory)

        # Pass through the feed-forward network
        return self.feed_forward(tgt)

## Full Transfomer Decoder

**Task**: implement the transfomer decoder part class

In [21]:
class TransformerDecoder(nn.Module):
    def __init__(
        self,
        num_layers: int = 6,
        dim_model: int = 512,
        num_heads: int = 8,
        dim_feedforward: int = 2048,
        dropout: float = 0.1,
    ):
        super().__init__()
        # Initialize laters from TransformerDecoderLayer instances
        self.layers = nn.ModuleList([
            TransformerDecoderLayer(dim_model, num_heads, dim_feedforward, dropout) for _ in range(num_layers)
        ])

    def forward(self, tgt: Tensor, memory: Tensor) -> Tensor:
        # Calculate sequence length and dimension from the target tensor
        seq_len, dimension = tgt.size(1), tgt.size(2)

        # Add position encoding to the target tensor
        tgt += position_encoding(seq_len, dimension, tgt.device)

        # Process each layer in the transformer decoder
        # Code here
        for layer in self.layers:
            tgt = layer(tgt, memory)

        # Apply a softmax to the output of the final linear layer
        return tgt

## Full Transfomer model

**Task**:
1. Assembly a full transfomer (Encoder + Decoder)
2. Implement the Transfomer training loop
3. Using dataset of your choice train the transformer just for one epoch


In [22]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, num_encoder_layers=12, num_decoder_layers=12, dim_model=512, num_heads=6, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, dim_model)
        self.encoder = TransformerEncoder(num_encoder_layers, dim_model, num_heads, dim_feedforward, dropout)
        self.decoder = TransformerDecoder(num_decoder_layers, dim_model, num_heads, dim_feedforward, dropout)
        # Define the output linear transformation
        self.linear = nn.Linear(dim_model, vocab_size)

    def forward(self, src, tgt):
        encoder_output = self.encoder(self.embedding(src))
        decoder_output = self.decoder(self.embedding(tgt), encoder_output)
        return self.linear(decoder_output)

In [23]:
from torch.utils.data import DataLoader, Dataset

class RandomDataset(Dataset):
    def __init__(self, num_samples, seq_length, vocab_size):
        self.num_samples = num_samples
        self.seq_length = seq_length
        self.vocab_size = vocab_size

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        src = torch.randint(1, self.vocab_size, (self.seq_length,))
        tgt = src.clone()
        return src, tgt

# Training loop
def train_transformer(model, dataloader, criterion, optimizer, device):
    model.train()
    for i in range(1):
        for batch_idx, (src, tgt) in enumerate(dataloader):
            src, tgt = src.to(device), tgt.to(device)
            optimizer.zero_grad()
            output = model(src, tgt[:, :-1])
            loss = criterion(output.view(-1, output.size(-1)), tgt[:, 1:].reshape(-1))
            loss.backward()
            optimizer.step()
            print(f"Batch {batch_idx}/{len(dataloader)}, Loss: {loss.item()}")

# Hyperparameters and setup
vocab_size = 10000
seq_length = 32
batch_size = 64
num_samples = 1000
learning_rate = 3e-4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Transformer(vocab_size).to(device)

dataset = RandomDataset(num_samples, seq_length, vocab_size)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train for one epoch
train_transformer(model, dataloader, criterion, optimizer, device)

Batch 0/16, Loss: 9.387690544128418
Batch 1/16, Loss: 9.376551628112793
Batch 2/16, Loss: 9.365616798400879
Batch 3/16, Loss: 9.403587341308594
Batch 4/16, Loss: 9.35863208770752
Batch 5/16, Loss: 9.378893852233887
Batch 6/16, Loss: 9.365923881530762
Batch 7/16, Loss: 9.362811088562012
Batch 8/16, Loss: 9.364035606384277
Batch 9/16, Loss: 9.369789123535156
Batch 10/16, Loss: 9.35463809967041
Batch 11/16, Loss: 9.356008529663086
Batch 12/16, Loss: 9.34257984161377
Batch 13/16, Loss: 9.379120826721191
Batch 14/16, Loss: 9.364068984985352
Batch 15/16, Loss: 9.33487319946289
